In [1]:
# # ============================================================
# # ITEM-LEVEL HABIT FEATURES (TF-IDF ANALOG)
# # ============================================================
# import numpy as np
# import pandas as pd

# def build_habit_features(df, tau_days=120):
#     df = df.copy()
#     df["date"] = pd.to_datetime(df["date"])

#     total_trips = df["date"].nunique()
#     timeline_days = (df["date"].max() - df["date"].min()).days or 1

#     rows = []

#     for itemId, g in df.groupby("itemId"):
#         buys = g[g["didBuy_target"] == 1]["date"]

#         if len(buys) == 0:
#             rows.append({
#                 "itemId": itemId,
#                 "habitFrequency_feat": 0.0,
#                 "habitSpan_feat": 0.0,
#                 "habitDecay_feat": 0.0,
#             })
#             continue

#         first = buys.min()
#         last = buys.max()

#         habitFrequency = len(buys) / total_trips
#         habitSpan = (last - first).days / timeline_days
#         days_since_last = (df["date"].max() - last).days
#         habitDecay = np.exp(-days_since_last / tau_days)

#         rows.append({
#             "itemId": itemId,
#             "habitFrequency_feat": habitFrequency,
#             "habitSpan_feat": habitSpan,
#             "habitDecay_feat": habitDecay,
#         })

#     return pd.DataFrame(rows)
# ###############################################################################


# def compute_due_score(df,itemId=None,use_sigmoid=True,normalize=False, weights=None):
#     """
#     """

#     if weights is None:
#         weights = {
#             "daysSinceLastPurchase_feat": 1.5,
#             "freq_30": 1.0,
#             "freq_90": 0.5
#         }

#     # --------------------------------------------------------
#     # Optional itemId filter
#     # --------------------------------------------------------
#     if itemId is not None:
#         df = df[df["itemId"] == itemId].copy()
#     else:
#         df = df.copy()

#     # --------------------------------------------------------
#     # RAW linear score (pre-normalization)
#     # --------------------------------------------------------
#     df["due_score_raw"] = (
#         weights["daysSinceLastPurchase_feat"] * df["daysSinceLastPurchase_feat"]
#       + weights["freq_30_feat"]              * df["freq_30_feat"]
#       + weights["freq_90_feat"]              * df["freq_90_feat"]
#     )

#     # --------------------------------------------------------
#     # Final due_score
#     # --------------------------------------------------------
#     if use_sigmoid:
#         df["due_score_feat"] = 1 / (1 + np.exp(-df["due_score_raw"]))

#     elif normalize:
#         mean = df["due_score_raw"].mean()
#         std  = df["due_score_raw"].std() or 1.0
#         df["due_score"] = (df["due_score_raw"] - mean) / std

#     else:
#         df["due_score"] = df["due_score_raw"]

#     return df
###############################################################################


# ============================================================
# MERGE HABIT FEATURES
# ============================================================
#habit_df = build_habit_features(combined_df)

#combined_df = combined_df.merge(habit_df, on="itemId",how="left")

#combined_df[["habitFrequency_feat", "habitSpan_feat", "habitDecay_feat"]] = (
#    combined_df[["habitFrequency_feat", "habitSpan_feat", "habitDecay_feat"]].fillna(0.0)
#)




In [2]:
#combined_df["purchaseToTripRatio"] = combined_df["daysSinceLastPurchase"] / combined_df["avgDaysBetweenPurchases"]


# encoded_df["due_score"] = (
#     1.5 * encoded_df["daysSinceLastPurchase_norm"]
#   + 1.0 * encoded_df["freq_30_norm"]
#   + 0.5 * encoded_df["freq_90_norm"]
# )

#encoded_df["due_score"] = 1 / (1 + np.exp(-encoded_df["due_score"]))

# df["bulkAdjustedUrgencyRatio"] = df.apply(
#     lambda row: BulkPurchaseFeatures.bulk_adjusted_urgency_ratio(
#         row["daysSinceLastPurchase"],
#         row["avgDaysBetweenPurchases"],
#         row["bulkFlag"],
#         row["didBuy"]          # this is already 1 or 0 per trip
#     ),
#     axis=1
# )


In [3]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime

from grocery_ml import GroceryML
from hidden_layer_param_builder import HiddenLayerParamSetBuilder

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

C:\Users\steve\source\repos\grocery-ml


In [4]:
def run_all_experiments(training_df, model_param_sets, output_dir):
    total = len(model_param_sets)
    print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
    for index, params in enumerate(model_param_sets, 1):
        print(f"Running Exp {index}/{total}...")
        groceryML.run_experiment(training_df, params["buildParams"], params["trainParams"], output_dir)

groceryML = GroceryML();
groceryML.build_combined_df()
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
groceryML.export_df_to_excel_table(groceryML.combined_df, f"groceryml-combined_df-{ts}", "combined_df")

insert_negative_samples()
build_habit_frequency_for_training()
compute_habit_frequency_map()
build_trip_level_features()
build_purchase_item_freq_cols()
Writing XLSX: groceryml-combined_df-2025_12_27_02_03_56.xlsx
   XLSX Done: groceryml-combined_df-2025_12_27_02_03_56.xlsx


In [ ]:
modelParamsList = []

# activations to try (no softmax)
output_activations = ["sigmoid", "linear", "tanh"]
# output_activations = ["sigmoid", "linear", "tanh", "elu"]
embDims = [12, 32, 64, 128]
          
for act in output_activations:
    for dim in embDims:
        # accuracy
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": [64,32],
                "activation": "relu",
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": ["accuracy"]
            }
        })
    
        # auc
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": [64,32],
                "activation": "relu",
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": ["AUC"]
            }
        })
    
        # precision + recall
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": [64,32],
                "activation": "relu",
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": ["Precision", "Recall"]
            }
        })
    
        # auc + precision + recall
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
               "embedding_dim": dim,
                "layers": [64,32],
                "activation": "relu",
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": ["AUC", "Precision", "Recall"]
            }
        })
    
        # mae
        modelParamsList.append({
            "trainParams": { "epochs": 20 },
            "buildParams": {
                "embedding_dim": dim,
                "layers": [64,32],
                "activation": "relu",
                "output_activation": act,
                "optimizer": "adam",
                "learning_rate": 0.001,
                "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                "metrics": ["mae"]
            }
    })

ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments(groceryML.combined_df, modelParamsList, f"exp/keras/{ts}")

run_all_experiments() when: 2025-12-27 02:06:19.736645  output_dir: exp/keras/2025_12_27_02_06_19
Running Exp 1/60...
run_experiment()  baseDir: exp/keras/2025_12_27_02_06_19  
run_experiment()  when: 2025-12-27 02:06:19.736645 params: {'epochs': 20}  
normalize_features()
train_model()
 build_prediction_input()   Prediction date: 2025-12-27
recompute_habit_frequency_for_prediction_time()
compute_habit_frequency_map()
normalize_features()
Running Model.Predict()
23/23 [==============================] - 0s 616us/step
Creating dir: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__162854
Exporting dataframes:
Writing XLSX: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__162854\predictions-e12__l64-32__ep20__oa_sigmoid__162854.xlsx
   XLSX Done: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__162854\predictions-e12__l64-32__ep20__oa_sigmoid__162854.xlsx
Writing CSV: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__162854\predictions-e12__l6

INFO:tensorflow:Assets written to: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__162854\model\assets


INFO:tensorflow:Assets written to: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__162854\model\assets


Saved experiment → exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__162854
Running Exp 2/60...
run_experiment()  baseDir: exp/keras/2025_12_27_02_06_19  
run_experiment()  when: 2025-12-27 02:12:28.079447 params: {'epochs': 20}  
normalize_features()
train_model()
 build_prediction_input()   Prediction date: 2025-12-27
recompute_habit_frequency_for_prediction_time()
compute_habit_frequency_map()
normalize_features()
Running Model.Predict()
23/23 [==============================] - 0s 641us/step
Creating dir: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__161566
Exporting dataframes:
Writing XLSX: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__161566\predictions-e12__l64-32__ep20__oa_sigmoid__161566.xlsx
   XLSX Done: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__161566\predictions-e12__l64-32__ep20__oa_sigmoid__161566.xlsx
Writing CSV: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__161566\predictions-e12__l64-32__ep20_

INFO:tensorflow:Assets written to: exp/keras/2025_12_27_02_06_19\e12__l64-32__ep20__oa_sigmoid__161566\model\assets


...

ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssoc

Creating dir: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065
Exporting dataframes:
Writing XLSX: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\predictions-e128__l64-32__ep20__oa_linear__191065.xlsx
   XLSX Done: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\predictions-e128__l64-32__ep20__oa_linear__191065.xlsx
Writing CSV: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\predictions-e128__l64-32__ep20__oa_linear__191065.csv
  CSV done: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\predictions-e128__l64-32__ep20__oa_linear__191065.csv
Writing XLSX: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\normalized_df-e128__l64-32__ep20__oa_linear__191065.xlsx


ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssoc

   XLSX Done: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\normalized_df-e128__l64-32__ep20__oa_linear__191065.xlsx
Writing CSV: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\normalized_df-e128__l64-32__ep20__oa_linear__191065.csv
  CSV done: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\normalized_df-e128__l64-32__ep20__oa_linear__191065.csv
Writing XLSX: exp/keras/2025_12_27_02_06_19\e128__l64-32__ep20__oa_linear__191065\combined_df-e128__l64-32__ep20__oa_linear__191065.xlsx


ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:asyncio:Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\ProgramData\miniconda3\envs\grocery-ml\lib\asyncio\selector_events.py", line 115, in _read_from_self
    data = self._ssoc